In [1]:
import numpy as np
import sys
import time
from clickhouse_driver import Client, connect

In [8]:
client = Client('localhost')

In [9]:
client.execute('show databases')

Failed to connect to localhost:9000
Traceback (most recent call last):
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 318, in connect
    return self._init_connection(host, port)
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 282, in _init_connection
    self.socket = self._create_socket(host, port)
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 254, in _create_socket
    raise err
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 245, in _create_socket
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused


NetworkError: Code: 210. Connection refused (localhost:9000)

In [10]:
client.execute('use bd')

[]

In [68]:
client.execute('show tables')

[]

In [30]:
import datetime
today = datetime.datetime.now()

(today.strftime("%Y-%m-%d %H:%M:%S"))
# str(today)

'2021-06-06 19:16:34'

In [15]:
def genDict(news = 'BBC'): 
    randList = np.random.dirichlet(np.ones(6),size=1)[0]
    maxList = [0]*6
    maxInd = np.argmax(randList)
    maxList[maxInd] = 1
    today = datetime.datetime.now()
    time = today.strftime("%Y-%m-%d %H:%M:%S")
    
    events = ["others", "sports", "crime", "covid", "Business", "Political"]
    eventDict = dict(zip(events, maxList))
    eventPred_dict = dict(zip(events, randList))
    finalDict = {"time": time, "channel": news ,"event": eventDict, "eventPred": eventPred_dict }
    # print(eventDict)
    # time.sleep(1)
    return finalDict

In [16]:
genDict()

{'time': '2021-06-06 22:43:36',
 'channel': 'BBC',
 'event': {'others': 0,
  'sports': 0,
  'crime': 0,
  'covid': 0,
  'Business': 1,
  'Political': 0},
 'eventPred': {'others': 0.041075786181733596,
  'sports': 0.27013367954693196,
  'crime': 0.2577574055278343,
  'covid': 0.03408552779598859,
  'Business': 0.30582440725657467,
  'Political': 0.09112319369093673}}

In [ ]:
client.execute("create table IF NOT EXISTS news(time DateTime('Europe/Moscow'), \
channel String, \
eventPred Nested\
(\
others Float32, \
sports Float32, \
crime Float32,\
covid Float32, \
Business Float32, \
Political Float32\
), \
event Nested\
(\
others UInt8, \
sports UInt8, \
crime UInt8,\
covid UInt8, \
Business UInt8, \
Political UInt8\
) \
ENGINE = MergeTree() ORDER BY time")

In [ ]:
client.execute("create table IF NOT EXISTS news_kafka(time DateTime('Europe/Moscow'), \
channel String, \
eventPred Nested\
(\
others Float32, \
sports Float32, \
crime Float32,\
covid Float32, \
Business Float32, \
Political Float32\
), \
event Nested\
(\
others UInt8, \
sports UInt8, \
crime UInt8,\
covid UInt8, \
Business UInt8, \
Political UInt8\
) \
ENGINE = Kafka('localhost:9092', 'news', 'Cgroup') \
SETTINGS kafka_format = 'JSONEachRow', kafka_row_delimiter = '\n';")")

In [ ]:
client.execute("CREATE MATERIALIZED VIEW bd.mv_news_kafka TO bd.news AS SELECT * FROM bd.news_kafka;")

In [17]:
while True:
    response = genDict()
    producer.send('news', value = response)
    print(response)
    time.sleep(1)

{'time': '2021-06-06 22:43:50', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 1}, 'eventPred': {'others': 0.05952426533355135, 'sports': 0.12403592593464254, 'crime': 0.2150439585806215, 'covid': 0.1348644835814176, 'Business': 0.2299475613243681, 'Political': 0.23658380524539893}}
{'time': '2021-06-06 22:43:51', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 1}, 'eventPred': {'others': 0.14024166070534821, 'sports': 0.2132038412437504, 'crime': 0.20310351874500662, 'covid': 0.02000662789057691, 'Business': 0.024427684979183406, 'Political': 0.3990166664361344}}
{'time': '2021-06-06 22:43:52', 'channel': 'BBC', 'event': {'others': 1, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.2682527531227151, 'sports': 0.20346719070938668, 'crime': 0.02740594768795189, 'covid': 0.23724227196212946, 'Business': 0.12766067818605709, 'P

{'time': '2021-06-06 22:44:14', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 1, 'Political': 0}, 'eventPred': {'others': 0.027538472791328886, 'sports': 0.27512527916223317, 'crime': 0.040751487007621046, 'covid': 0.052789223337431794, 'Business': 0.5567545686200471, 'Political': 0.047040969081337945}}
{'time': '2021-06-06 22:44:15', 'channel': 'BBC', 'event': {'others': 1, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.4892125787169421, 'sports': 0.09101379108845858, 'crime': 0.0862637469327807, 'covid': 0.24180833245417244, 'Business': 0.030834053934122967, 'Political': 0.06086749687352306}}
{'time': '2021-06-06 22:44:16', 'channel': 'BBC', 'event': {'others': 1, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.36495851873179663, 'sports': 0.012136736174855205, 'crime': 0.24297589492778973, 'covid': 0.07442300417327916, 'Business': 0.1476083577702

{'time': '2021-06-06 22:44:38', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 1, 'Political': 0}, 'eventPred': {'others': 0.05259526466601368, 'sports': 0.13292961533988032, 'crime': 0.17281059845565375, 'covid': 0.23712994722474756, 'Business': 0.339728263015294, 'Political': 0.06480631129841065}}
{'time': '2021-06-06 22:44:39', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 1, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.0020463576185520116, 'sports': 0.020578225589921284, 'crime': 0.41014372709185987, 'covid': 0.1669742395937116, 'Business': 0.3868871158077043, 'Political': 0.01337033429825098}}
{'time': '2021-06-06 22:44:40', 'channel': 'BBC', 'event': {'others': 1, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.37012364997854974, 'sports': 0.32989525720849355, 'crime': 0.14613005013542657, 'covid': 0.025509032514608097, 'Business': 0.0068091666388845

{'time': '2021-06-06 22:45:02', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 1, 'Political': 0}, 'eventPred': {'others': 0.13785747828967093, 'sports': 0.04544001465483671, 'crime': 0.3105609883175568, 'covid': 0.031136337683626916, 'Business': 0.37509619803796906, 'Political': 0.09990898301633959}}
{'time': '2021-06-06 22:45:03', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 1, 'Political': 0}, 'eventPred': {'others': 0.005594132982257059, 'sports': 0.33143499644882124, 'crime': 0.08633642521709693, 'covid': 0.08072362297516975, 'Business': 0.41793862287520905, 'Political': 0.07797219950144609}}
{'time': '2021-06-06 22:45:04', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 1, 'Political': 0}, 'eventPred': {'others': 0.2488404300220564, 'sports': 0.09999547381472182, 'crime': 0.01067221404684236, 'covid': 0.2330980826288222, 'Business': 0.31726758498917274

{'time': '2021-06-06 22:45:26', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 1, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.2434409686102145, 'sports': 0.033538411967143, 'crime': 0.004143801130391508, 'covid': 0.5998959005622077, 'Business': 0.049043982183731356, 'Political': 0.06993693554631204}}
{'time': '2021-06-06 22:45:27', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 1, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.1444953005513544, 'sports': 0.11118450969141927, 'crime': 0.35140695614342865, 'covid': 0.024426045646064837, 'Business': 0.23784427135718927, 'Political': 0.13064291661054356}}
{'time': '2021-06-06 22:45:28', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 1}, 'eventPred': {'others': 0.043993727606493456, 'sports': 0.2089580120942755, 'crime': 0.11379608101777662, 'covid': 0.12270776624326525, 'Business': 0.23812280796661175,

{'time': '2021-06-06 22:45:50', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 1, 'Political': 0}, 'eventPred': {'others': 0.22469503344070652, 'sports': 0.09486598042991583, 'crime': 0.07904630530551957, 'covid': 0.13361910348137682, 'Business': 0.28131325457354056, 'Political': 0.18646032276894092}}
{'time': '2021-06-06 22:45:51', 'channel': 'BBC', 'event': {'others': 0, 'sports': 1, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 0}, 'eventPred': {'others': 0.07424963457963403, 'sports': 0.2718482402469946, 'crime': 0.17248213679016455, 'covid': 0.109103168453665, 'Business': 0.11190911251312256, 'Political': 0.2604077074164193}}
{'time': '2021-06-06 22:45:52', 'channel': 'BBC', 'event': {'others': 0, 'sports': 0, 'crime': 0, 'covid': 0, 'Business': 0, 'Political': 1}, 'eventPred': {'others': 0.08045033002723766, 'sports': 0.34420039462269514, 'crime': 0.07283056529446372, 'covid': 0.016339302185356955, 'Business': 0.13138060583859867, 

KeyboardInterrupt: 

In [12]:
import json
import datetime
import numpy as np
import sys
import time
from kafka import KafkaProducer
from collections import OrderedDict
producer = KafkaProducer(bootstrap_servers = 'localhost:9092',value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [19]:
client.execute("INSERT INTO bd.news format JSONEachRow {}".format(genDict()))

Failed to connect to localhost:9000
Traceback (most recent call last):
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 318, in connect
    return self._init_connection(host, port)
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 282, in _init_connection
    self.socket = self._create_socket(host, port)
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 254, in _create_socket
    raise err
  File "/home/chaitanya/anaconda3/lib/python3.8/site-packages/clickhouse_driver/connection.py", line 245, in _create_socket
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused


NetworkError: Code: 210. Connection refused (localhost:9000)